In [5]:
pip install cmake

Note: you may need to restart the kernel to use updated packages.


ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\CHAITANYA\anaconda3\lib\site-packages\pip\_vendor\urllib3\response.py", line 438, in _error_catcher
    yield
  File "C:\Users\CHAITANYA\anaconda3\lib\site-packages\pip\_vendor\urllib3\response.py", line 519, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "C:\Users\CHAITANYA\anaconda3\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 62, in read
    data = self.__fp.read(amt)
  File "C:\Users\CHAITANYA\anaconda3\lib\http\client.py", line 463, in read
    n = self.readinto(b)
  File "C:\Users\CHAITANYA\anaconda3\lib\http\client.py", line 507, in readinto
    n = self.fp.readinto(b)
  File "C:\Users\CHAITANYA\anaconda3\lib\socket.py", line 704, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\CHAITANYA\anaconda3\lib\ssl.py", line 1241, in recv_into
    return self.read(nbytes, buffer)
  File "C:\Users\CHAITANYA\anaconda3\lib\ssl.py", line 1099, in read
    return se

In [3]:
import cv2
import face_recognition
import os
import numpy as np

known_faces_dir = 'Images'

known_face_encodings = []
known_face_names = []

for filename in os.listdir(known_faces_dir):
    if not filename.startswith('.'):  
        img = cv2.imread(os.path.join(known_faces_dir, filename))
        if img is not None:
            known_face_encodings.append(face_recognition.face_encodings(img)[0])
            name = os.path.splitext(filename)[0]
            known_face_names.append(name)
        else:
            print(f"Error loading image: {filename}")

def detect_eye_blink(frame, face_location):
    top, right, bottom, left = face_location
    face_roi = frame[top:bottom, left:right]
    gray = cv2.cvtColor(face_roi, cv2.COLOR_BGR2GRAY)
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    return len(eyes) >= 2

def detect_open_mouth(face_landmarks, prev_open_mouth):
    top_lip_bottom = min(face_landmarks['top_lip'], key=lambda x: x[1])[1]
    bottom_lip_top = max(face_landmarks['bottom_lip'], key=lambda x: x[1])[1]
    mouth_height = bottom_lip_top - top_lip_bottom
    threshold = 60
    return prev_open_mouth or mouth_height > threshold

video_capture = cv2.VideoCapture(0)

blink_count = 0
instruction_text = "Please blink 3 times and open your mouth for liveness detection..."
open_mouth = False 
while True:
    ret, frame = video_capture.read()
    if not ret:
        print("Error: Failed to capture frame from webcam.")
        break
    cv2.putText(frame, instruction_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 2)
    cv2.putText(frame, f"Blink Count: {blink_count}", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    face_locations = face_recognition.face_locations(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)
    face_landmarks_list = face_recognition.face_landmarks(frame, face_locations)

    for face_encoding, face_location, landmarks in zip(face_encodings, face_locations, face_landmarks_list):
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        face_distance = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distance)

        if matches[best_match_index]:
            name = known_face_names[best_match_index]
            open_mouth = detect_open_mouth(landmarks, open_mouth)
            eye_blink = detect_eye_blink(frame, face_location)
            
            if open_mouth:
                cv2.putText(frame, "Mouth: Open", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            else:
                cv2.putText(frame, "Mouth: Closed", (50, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            for (x, y) in landmarks['top_lip']:
                cv2.circle(frame, (x, y), 1, (255, 255, 255), -1)
            for (x, y) in landmarks['bottom_lip']:
                cv2.circle(frame, (x, y), 1, (255, 255, 255), -1)
        
            cv2.rectangle(frame, (face_location[3], face_location[0]), (face_location[1], face_location[2]), (0, 255, 0), 2)
            font = cv2.FONT_HERSHEY_COMPLEX
            if blink_count >= 3 and open_mouth:
                cv2.putText(frame, f"{name} (Real)", (face_location[3] + 6, face_location[2] - 6), font, 1, (0, 255, 0), 2)
            else:
                cv2.putText(frame, f"{name} (Spoof)", (face_location[3] + 6, face_location[2] - 6), font, 1, (0, 0, 255), 2)

            if not eye_blink:
                blink_count += 1

    cv2.imshow('Face Recognition', frame)

    key = cv2.waitKey(1)
    if key & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'face_recognition'

In [2]:
pip install face_recognition


  Using cached face_recognition-1.3.0-py2.py3-none-any.whl (15 kB)
  Using cached face_recognition_models-0.3.0-py2.py3-none-any.whl
  Using cached dlib-19.24.2.tar.gz (11.8 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
Failed to build dlib
Note: you may need to restart the kernel to use updated packages.


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\CHAITANYA\anaconda3\python.exe' 'C:\Users\CHAITANYA\anaconda3\lib\site-packages\pip\_vendor\pep517\in_process\_in_process.py' build_wheel 'C:\Users\CHAITA~1\AppData\Local\Temp\tmpgq976rab'
       cwd: C:\Users\CHAITANYA\AppData\Local\Temp\pip-install-41jrs1ih\dlib_22f319f913d74a72b68684c979a4bc72
  Complete output (73 lines):
  running bdist_wheel
  running build
  running build_ext
  <string>:125: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  Building extension for Python 3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]
  Invoking CMake setup: 'cmake C:\Users\CHAITANYA\AppData\Local\Temp\pip-install-41jrs1ih\dlib_22f319f913d74a72b68684c979a4bc72\tools\python -DCMAKE_LIBRARY_OUTPUT_DIRECTORY=C:\Users\CHAITANYA\AppData\Local\Temp\pip-install-41jrs1ih\dlib_22f319f913d74a72b68684c979a4bc72\build\lib.win-amd64-cpython-39 -DPYTHON_EXECUTABLE=C:\Users\CHAITA